In [857]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import requests
import pprint

teams = [
#     
      #"BOS",  # Boston Bruins
      #"ANA",  # Anaheim Ducks
#      "BUF",  # Buffalo Sabres
#    "CAR",  # Carolina Hurricanes
 #    "CBJ",  # Columbus Blue Jackets
#     "CGY",  # Calgary Flames
#     "CHI",  # Chicago Blackhawks
#     "COL",  # Colorado Avalanche
#     "DAL",  # Dallas Stars
#     "DET",  # Detroit Red Wings
                                    #"EDM",  # Edmonton Oilers
#     "FLA",  # Florida Panthers
#     "LAK",  # Los Angeles Kings
#     "MIN",  # Minnesota Wild
#     "MTL",  # Montreal Canadiens
#     "NJD",  # New Jersey Devils
#     "NSH",  # Nashville Predators
#     "NYI",  # New York Islanders
                                    #"NYR",  # New York Rangers
#     "OTT",  # Ottawa Senators
#     "PHI",  # Philadelphia Flyers
#     "PIT",  # Pittsburgh Penguins
#     "SEA",  # Seattle Kraken
#     "SJS",  # San Jose Sharks
#     "STL",  # St. Louis Blues
#     "TBL",  # Tampa Bay Lightning
#     "TOR",  # Toronto Maple Leafs
#     "UTA",  # Utah Hockey Club
#      "VAN",  # Vancouver Canucks
#     "VGK",  # Vegas Golden Knights
#     "WPG",  # Winnipeg Jets
#     "WSH"   # Washington Capitals
#      "ARI"   # Arizona Coyotes
#     "ATL"   # Atlanta Thrashers 
#     "HFD"   # Hartford
#     "PHX"   # Pheonix
#     "QUE"   # Quebec
#     "WIN"   # Winnepeg Jets - Old
    ]

seasons = [

    # "19701971", "19711972", "19721973", "19731974", "19741975",
    # "19751976", "19761977", "19771978", "19781979", "19791980",
    # "19801981", "19811982", "19821983", "19831984", "19841985",
    # "19851986", "19861987", "19871988", "19881989", "19891990",
    # "19901991", "19911992", "19921993", "19931994", "19941995",
    # "19951996", "19961997", "19971998", "19981999", "19992000",
    # "20002001", "20012002", "20022003", "20032004", "20042005",
    # "20052006", "20062007", "20072008", "20082009", "20092010",
    # "20102011", "20112012", "20122013", "20132014", "20142015",
    # "20152016", "20162017", "20172018", "20182019", "20192020",
    # "20202021", "20212022", "20222023", "20232024", "20242025"
]

# Function to get player data from the NHL API
def get_player_data(team, season):
    url = f"https://api-web.nhle.com/v1/roster/{team}/{season}"
    response = requests.get(url)

    # Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for team {team} in season {season}")
        return {}

# Initialize a dictionary to store player IDs
player_ids = {}

# Loop through each season in the seasons list
for season in seasons:
    player_ids[season] = {}

    # Loop through each team
    for team in teams:
        try:
            # Get the player data for the team and season
            player_data = get_player_data(team, season)
        
            # Initialize dictionaries for defencemen, forwards, and goalies
            player_ids[season][team] = {
                #'defencemen': [],
                'forwards': [],
                #'goalies': []
            }
        
            # # Extract player IDs from defencemen
            # if 'defensemen' in player_data:
            #     for player in player_data['defensemen']:
            #         player_id = player['id']
            #         player_ids[season][team]['defencemen'].append(player_id)
        
            # Extract player IDs from forwards
            if 'forwards' in player_data:
                for player in player_data['forwards']:
                    player_id = player['id']
                    player_ids[season][team]['forwards'].append(player_id)
        
            # # Extract player IDs from goalies
            # if 'goalies' in player_data:
            #     for player in player_data['goalies']:
            #         player_id = player['id']
            #         player_ids[season][team]['goalies'].append(player_id)
    
        except KeyError:
            print(f"Team {team} not found for season {season}. Skipping...")
            continue

#Print the player IDs
pprint.pprint(player_ids)

In [859]:
# Function to get game logs for a specific season
def get_game_log(player_id, season):
    url = f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2"
    response = requests.get(url)
    return response.json()

# Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for player {player_id} in season {season}")
        return {}

In [860]:
import pandas as pd

# Function to aggregate game logs for a specific player_id
def aggregate_game_logs(game_logs):
    aggregated_data = {}
    
    for game in game_logs:
        for key, value in game.items():
            if key not in aggregated_data:
                aggregated_data[key] = value
            else:
                # If the key already exists, aggregate the data (e.g., sum the values)
                if isinstance(value, (int, float)):
                    aggregated_data[key] += value
                elif isinstance(value, list):
                    aggregated_data[key].extend(value)
                # Handle other data types as needed
    
    return aggregated_data



# Initialize a dictionary to store DataFrames for each player_id
player_dfs = {}

# Loop through the selected player_ids
for season, teams in player_ids.items():
    for team, players in teams.items():
        for player_type, player_ids in players.items():
            for player_id in player_ids:
                game_log = get_game_log(player_id, season)
                
                if game_log and 'gameLog' in game_log:
                    game_log_data = game_log['gameLog']

                    seasons_list = []
                    
                    for stat in game_log_data:
                        stats_info = {
                            'gameDate': stat.get('gameDate'),
                            'homeRoadFlag': stat.get('homeRoadFlag'),    
                            'goals': stat.get('goals'),    
                            'assists': stat.get('assists'),    
                            'points': stat.get('points'),    
                            'plusMinus': stat.get('plusMinus'),    
                            'powerPlayGoals': stat.get('powerPlayGoals'),    
                            'powerPlayPoints': stat.get('powerPlayPoints'),    
                            'gameWinningGoals': stat.get('gameWinningGoals'),    
                            'otGoals': stat.get('otGoals'),    
                            'shots': stat.get('shots'),    
                            'shifts': stat.get('shifts'),        
                            'pim': stat.get('pim')
                        }
                        seasons_list.append(stats_info)
                    
                    # Create df from the flattened list of data
                    player_df = pd.DataFrame(seasons_list)

                    #  # Set gameDate as the index
                    # player_df.set_index('gameDate', inplace=True)

                    # Add the DataFrame to the player_dfs dictionary
                    player_dfs[player_id] = player_df

  
                   # Save the DataFrame to a CSV file
                    csv_filename = f'player_{player_id}.csv'
                    player_df.to_csv(csv_filename)
                    print(f"DataFrame for player_id {player_id} saved to {csv_filename}")
                else:
                    print(f"No valid data found for player_id {player_id} in season {season}")




DataFrame for player_id 8467906 saved to player_8467906.csv
DataFrame for player_id 8471863 saved to player_8471863.csv
DataFrame for player_id 8470719 saved to player_8470719.csv
DataFrame for player_id 8470358 saved to player_8470358.csv
DataFrame for player_id 8470708 saved to player_8470708.csv
DataFrame for player_id 8470854 saved to player_8470854.csv
DataFrame for player_id 8471498 saved to player_8471498.csv
DataFrame for player_id 8470274 saved to player_8470274.csv
DataFrame for player_id 8474570 saved to player_8474570.csv
DataFrame for player_id 8470616 saved to player_8470616.csv
DataFrame for player_id 8470654 saved to player_8470654.csv
DataFrame for player_id 8467334 saved to player_8467334.csv
DataFrame for player_id 8471268 saved to player_8471268.csv
DataFrame for player_id 8469584 saved to player_8469584.csv
DataFrame for player_id 8471664 saved to player_8471664.csv
DataFrame for player_id 8471628 saved to player_8471628.csv
DataFrame for player_id 8467463 saved to

In [861]:
# Function to load all CSV files into a dictionary of DataFrames
def load_all_csv_files(directory):
    csv_files = glob.glob(os.path.join(directory, 'player_*.csv'))
    dataframes = {}
    
    for csv_file in csv_files:
        player_id = os.path.basename(csv_file).split('_')[1].split('.')[0]
        df = pd.read_csv(csv_file)
        dataframes[player_id] = df
    
    return dataframes

# Specify the directory where the CSV files are saved
directory = '.'  # Current directory

# Load all CSV files into a dictionary of DataFrames
player_dfs = load_all_csv_files(directory)

# Print the loaded DataFrames for each player_id
#for player_id, df in player_dfs.items():
    # print(f"DataFrame for player_id {player_id}:")
    # print(df.head())

In [862]:
# Function to create a new DataFrame grouped by homeRoadFlag
def create_grouped_df(df):
    grouped_df = df.groupby('homeRoadFlag').agg({
        'gameDate': 'count',
        'goals': 'sum',
        'assists': 'sum',
        'points': 'sum',
        'plusMinus': 'sum',
        'powerPlayGoals': 'sum',
        'powerPlayPoints': 'sum',
        'gameWinningGoals': 'sum',
        'otGoals': 'sum',
        'shots': 'sum',
        'shifts': 'sum',
        'pim': 'sum',
        
    }).rename(columns={'gameDate': 'gameCount'})
    # Sum the grouped statistics to create a single row DataFrame
    single_row_df = grouped_df.sum().to_frame().T
    
    return single_row_df

In [863]:
# Loop through each DataFrame and create a grouped DataFrame
grouped_dfs = {}
for player_id, df in player_dfs.items():
    try:
        # Check if 'gameDate' column has values
        if df['gameDate'].empty:
            print(f"No gameDate values found for player_id {player_id}. Skipping...")
            continue

        # Check if 'homeRoadFlag' column exists
        if 'homeRoadFlag' in df.columns:
            grouped_df = create_grouped_df(df)
            grouped_dfs[player_id] = grouped_df
        else:
            print(f"'homeRoadFlag' column not found for player_id {player_id}. Skipping...")
    except Exception as e:
        print(f"An error occurred for player_id {player_id}: {e}")

An error occurred for player_id 8445144: 'gameDate'
An error occurred for player_id 8449583: 'gameDate'
An error occurred for player_id 8484136: 'gameDate'
An error occurred for player_id 8451939: 'gameDate'
An error occurred for player_id 8444933: 'gameDate'
An error occurred for player_id 8448499: 'gameDate'
An error occurred for player_id 8481712: 'gameDate'
An error occurred for player_id 8483476: 'gameDate'
An error occurred for player_id 8451109: 'gameDate'
An error occurred for player_id 8482691: 'gameDate'
An error occurred for player_id 8456602: 'gameDate'
An error occurred for player_id 8481576: 'gameDate'
An error occurred for player_id 8482132: 'gameDate'
An error occurred for player_id 8445889: 'gameDate'
An error occurred for player_id 8482673: 'gameDate'
An error occurred for player_id 8484149: 'gameDate'
An error occurred for player_id 8474053: 'gameDate'
An error occurred for player_id 8471698: 'gameDate'
An error occurred for player_id 8446541: 'gameDate'
An error occ

In [864]:
# Concatenate all DataFrames into a master DataFrame
master_df = pd.concat(grouped_dfs.values(), keys=grouped_dfs.keys(), names=['player_id', 'index'])

# Reset the index to have 'player_id' as a column
master_df.reset_index(level='index', drop=True, inplace=True)
master_df.to_csv('Vancouver-5.csv')

# #Print the master DataFrame
# print("Master DataFrame:")
# master_df.head()

In [866]:
# Load in fragmented csv
NYR_1 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_10_14.csv')
NYR_11 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_15_19.csv')
NYR_12 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_23.csv')
NYR_13 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_24.csv')
NYR_14 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_95_99.csv')
NYR_15 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_split_1.csv')
NYR_16 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Rangers_split.csv')



In [869]:
# Concatenate all DataFrames into one
rangers_df = pd.concat([NYR_1, NYR_11, NYR_12, NYR_13, NYR_14, NYR_15, NYR_16], ignore_index=True)

In [870]:
# Save the merged DataFrame to a CSV file
rangers_df.to_csv('Rangers.csv', index=False)
print("Merged DataFrame saved to Rangers.csv")

Merged DataFrame saved to Rangers.csv


In [867]:
EDM_1 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_1.csv')
EDM_2 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_2.csv')
EDM_3 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_3.csv')
EDM_4 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_4.csv')
EDM_5 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_5.csv')
EDM_6 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_6.csv')
EDM_7 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_14-19.csv')
EDM_8 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_15-20.csv')
EDM_9 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Edmonton_20-24.csv')

In [ ]:
# Concatenate all DataFrames into one
rangers_df = pd.concat([EDM_1, EDM_2, EDM_3, EDM_4, EDM_5, EDM_6, EDM_7, EDM_8, EDM_9], ignore_index=True)
# Save the merged DataFrame to a CSV file
rangers_df.to_csv('Edmonton.csv', index=False)
print("Merged DataFrame saved to Edmonton.csv")

Merged DataFrame saved to Rangers.csv


In [868]:
Van_1 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Vancouver-1.csv')
Van_2 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Vancouver-2.csv')
Van_3 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Vancouver-3.csv')
Van_4 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Vancouver-4.csv')
Van_5 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards/Vancouver-5.csv')

In [872]:
# Concatenate all DataFrames into one
rangers_df = pd.concat([Van_1, Van_2, Van_3, Van_4, Van_5], ignore_index=True)
# Save the merged DataFrame to a CSV file
rangers_df.to_csv('Vancouver.csv', index=False)
print("Merged DataFrame saved to Vancouver.csv")

Merged DataFrame saved to Vancouver.csv


In [873]:
import pandas as pd
import glob
import os

# Function to load and merge all CSV files from a specified directory into one DataFrame
def merge_all_csv_files(directory):
    csv_files = glob.glob(os.path.join(directory, '*.csv'))
    dataframes = []
    
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dataframes.append(df)
    
    # Concatenate all DataFrames into one
    merged_df = pd.concat(dataframes, ignore_index=True)
    
    return merged_df

# Specify the directory where the CSV files are saved
directory = '/Users/blairjdaniel/lighthouse/lighthouse/NHL/csv_files_forwards'

# Merge all CSV files into one DataFrame
master_df = merge_all_csv_files(directory)

# Save the merged DataFrame to a CSV file
master_df.to_csv('Master.csv', index=False)
print("Merged DataFrame saved to Master.csv")

Merged DataFrame saved to Master.csv


In [875]:
all_players =  pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/Master.csv')
len(all_players)

169588

In [876]:
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169588 entries, 0 to 169587
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   player_id         169588 non-null  int64  
 1   gameCount         169588 non-null  float64
 2   goals             169588 non-null  float64
 3   assists           169588 non-null  float64
 4   points            169588 non-null  float64
 5   plusMinus         169588 non-null  float64
 6   powerPlayGoals    169588 non-null  float64
 7   powerPlayPoints   169588 non-null  float64
 8   gameWinningGoals  169588 non-null  float64
 9   otGoals           169588 non-null  float64
 10  shots             169588 non-null  float64
 11  shifts            169588 non-null  float64
 12  pim               169588 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 16.8 MB


In [880]:
all_players.describe()

count     169588
unique         2
top        False
freq      127997
Name: gameCount, dtype: object

In [879]:
all_players = all_players['gameCount'] > 60
len(all_players)

169588